
<br>
-------------------------------------------------------------------------------<br>
Name:        energeticEvaluations<br>
Purpose:     Evaluation of energetic impact of HVAC system based on provided static data set<br>
Author:      Marcus Vogt<br>
Created:     27.11.2021<br>
Copyright:   Chair of Sustainable Manufacturing and Life Cycle Engineering, Institute of Machine Tools and Production Technology, Technische UniversitÃ¤t Braunschweig, Langer Kamp 19b, 38106 Braunschweig, Germany<br>
Licence:     MIT (see License)<br>
-------------------------------------------------------------------------------<br>


In [13]:
import energeticEvaluations as enEval
import os
import pandas as pd

In [14]:
########### read in data set ############
current_dir = os.getcwd()
staticResultPath = os.path.join(current_dir, "data", "automateSimulationStaticResults.csv")
dfStaticResults = pd.read_csv(staticResultPath, index_col=0)

########### boundary parameters ############
consideredLocation = "OS" # considered available location without case number e.g. "OS"
useBoundaryParameters = True
if useBoundaryParameters:
  maxHumansInAirFlow = 3 # maximum number of humans in air flow zones in room
  maxHumansInRoom = 6 # total maximum amount of humans in room
  maxWasteHeatRoomW = 3000 # max waste heat in W
  roomDewPointDegrees = -50 # dew point temperature in °C
  inletDewPointDegrees = -60 # dew point temperature in °C
  inletTemperatureDegrees = 20 # inlet temperature in °C
  leakagesSuctionVolumeFlowM3H = 100 # volume flow in m^3/h
else: # this approach can be used if the boundary conditions below are known
  scalingFactorS = 1.1 # scaling factor calculated from necessary air flow
  maxMoistureLoad = 0.5 # max. moisture load in kg/s
  maxWasteHeatRoomW = 3000  # max waste heat in W

In [12]:
########### evaluations ############
if useBoundaryParameters:
  scalingFactorS, maxMoistureLoad = enEval.boundaryParameters2scalingFactorMoistureLoad(maxHumansInAirFlow= maxHumansInAirFlow,
                                                                             maxHumansInRoom=maxHumansInRoom,
                                                                             roomDewPointDegrees=roomDewPointDegrees,
                                                                             inletDewPointDegrees=inletDewPointDegrees,
                                                                             inletTemperatureDegrees=inletTemperatureDegrees,
                                                                             leakagesSuctionVolumeFlowM3H=leakagesSuctionVolumeFlowM3H)

print("The scaling factor S is: {}, volume flow is: {} m^3/h, max. moisture load is: {} kg/s"
    .format(round(scalingFactorS, 3), round(scalingFactorS*11000, 3), round(maxMoistureLoad, 7)))

finalResultsOfInterpolation = enEval.interpolateScalingFactorAndInternalLoads(dfStaticResults,
                            consideredLocation=consideredLocation, scalingFactorS=scalingFactorS,
                            maxWasteHeatRoomW=maxWasteHeatRoomW, maxMoistureLoad=maxMoistureLoad)
print("The final results of the interpolation are:")
print(finalResultsOfInterpolation)

The scaling factor S is: 0.963, volume flow is: 10597.58 m^3/h, max. moisture load is: 0.0001883 kg/s
The final results of the interpolation are:
ScalingFactorS                               0.963416
LoadScalingFactor                            1.538687
electricEnergyKwh                       380007.624359
naturalGasEnergyKwh                     511507.165703
districtHeatingEnergyKwh                136475.642148
volumeFlowMainVentilatorM3H                  2.950864
PowerDemandMainVentilator                 7492.109431
TemperatureBeforeMainVentilator             296.72828
TemperatureAfterMainVentilator             297.377458
volumeFlowRegenerationVentilatorM3H          0.884031
PowerDemandRegenerationVentilator         1406.339276
TemperatureRegenerationHeater              397.735016
HeatFlowRegenerationHeater               79981.782181
HeatFlowPreCooler/Heater-amb-1            7402.526662
TemperaturePreCooler-amb-1                     283.15
HeatFlowPreCooler-3-4                       